<a href="https://colab.research.google.com/github/jianzhiw/Deep-Learning/blob/master/TensorFlow/TensorFlow05_Save_and_Restore_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q h5py pyyaml
!pip install -q tf_nightly

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

W0818 03:34:08.305088 140051597113216 __init__.py:689] 

  TensorFlow's `tf-nightly` package will soon be updated to TensorFlow 2.0.

  Please upgrade your code to TensorFlow 2.0:
    * https://www.tensorflow.org/beta/guide/migration_guide

  Or install the latest stable TensorFlow 1.X release:
    * `pip install -U "tensorflow==1.*"`

  Otherwise your code may be broken by the change.

  


'1.15.0-dev20190817'

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


# Define a model #
Let's build a simple model we'll use to demonstrate saving and loading weights.

In [0]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()
model.summary()

W0818 03:35:35.668795 140051597113216 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


# Save checkpoints during training #
The primary use case is to automatically save checkpoints during and at the end of training. This way you can use a trained model without having to retrain it, or pick-up training where you left of—in case the training process was interrupted.
<br></br>
***tf.keras.callbacks.ModelCheckpoint*** is a callback that performs this task. The callback takes a couple of arguments to configure checkpointing.
<br></br>
## Checkpoint callback usage ##
Train the model and pass it the ModelCheckpoint callback:

In [0]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 960/1000 [===========================>..] - ETA: 0s - loss: 1.1627 - acc: 0.6552
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 393us/sample - loss: 1.1406 - acc: 0.6600 - val_loss: 0.7428 - val_acc: 0.7660
Epoch 2/10
 736/1000 [=====================>........] - ETA: 0s - loss: 0.4207 - acc: 0.8832
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 240us/sample - loss: 0.4185 - acc: 0.8830 - val_loss: 0.5388 - val_acc: 0.8230
Epoch 3/10
 704/1000 [====================>.........] - ETA: 0s - loss: 0.3029 - acc: 0.9119
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 244us/sample - loss: 0.2892 - acc: 0.9190 - val_loss: 0.4796 - val_acc: 0.8470
Epoch 4/10
 704/1000 [====================>.........] - ETA: 0s - loss: 0.2134 - acc: 0.9474
Epoch 00004: saving model to training_1/cp.ckpt
1000/10

In [0]:
!ls {checkpoint_dir}

checkpoint		     cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index


Create a new, untrained model. When restoring a model from only weights, you must have a model with the same architecture as the original model. Since it's the same model architecture, we can share weights despite that it's a different instance of the model.
<br></br>
Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [0]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 100us/sample - loss: 2.3226 - acc: 0.1210
Untrained model, accuracy: 12.10%


In [0]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 72us/sample - loss: 0.4085 - acc: 0.8690
Restored model, accuracy: 86.90%


# Checkpoint callback options #
The callback provides several options to give the resulting checkpoints unique names, and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every 5-epochs:

In [0]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)

W0818 03:44:56.939801 140051597113216 callbacks.py:864] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [0]:
!ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.data-00001-of-00002
cp-0000.ckpt.data-00000-of-00002  cp-0025.ckpt.index
cp-0000.ckpt.data-00001-of-00002  cp-0030.ckpt.data-00000-of-00002
cp-0000.ckpt.index		  cp-0030.ckpt.data-00001-of-00002
cp-0005.ckpt.data-00000-of-00002  cp-0030.ckpt.index
cp-0005.ckpt.data-00001-of-00002  cp-0035.ckpt.data-00000-of-00002
cp-0005.ckpt.index		  cp-0035.ckpt.data-00001-of-00002
cp-0010.ckpt.data-00000-of-00002  cp-0035.ckpt.index
cp-0010.ckpt.data-00001-of-00002  cp-0040.ckpt.data-00000-of-00002
cp-0010.ckpt.index		  cp-0040.ckpt.data-00001-of-00002
cp-0015.ckpt.data-00000-of-00002  cp-0040.ckpt.index
cp-0015.ckpt.data-00001-of-00002  cp-0045.ckpt.data-00000-of-00002
cp-0015.ckpt.index		  cp-0045.ckpt.data-00001-of-00002
cp-0020.ckpt.data-00000-of-00002  cp-0045.ckpt.index
cp-0020.ckpt.data-00001-of-00002  cp-0050.ckpt.data-00000-of-00002
cp-0020.ckpt.index		  cp-0050.ckpt.data-00001-of-00002
cp-0025.ckpt.data-00000-of-00002  cp-0050.ckpt.index


In [0]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [0]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

W0818 03:46:19.130409 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0818 03:46:19.132050 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0818 03:46:19.133179 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0818 03:46:19.135023 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0818 03:46:19.137620 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0818 03:46:19.139543 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.kernel
W0818 03:46:19.141170 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.bias
W0818 03:46:19.142306 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-1.kernel
W0

1000/1000 [==============================] - 0s 95us/sample - loss: 0.5030 - acc: 0.8660
Restored model, accuracy: 86.60%


# What are these files? #
The above code stores the weights to a collection of checkpoint-formatted files that contain only the trained weights in a binary format. Checkpoints contain: **One or more shards that contain your model's weights.** An index file that indicates which weights are stored in a which shard.
<br></br>
If you are only training a model on a single machine, you'll have one shard with the suffix: .data-00000-of-00001

# Manually save weights #
Above you saw how to load the weights into a model.
<br></br>
Manually saving the weights is just as simple, use the ***Model.save_weights*** method.

In [0]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 95us/sample - loss: 0.5030 - acc: 0.8660
Restored model, accuracy: 86.60%


# Save the entire model #
The entire model can be saved to a file that contains the weight values, the model's configuration, and even the optimizer's configuration (depends on set up). This allows you to checkpoint a model and resume training later—from the exact same state—without access to the original code.
<br></br>
Saving a fully-functional model is very useful—you can load them in TensorFlow.js ([HDF5](https://www.hdfgroup.org/solutions/hdf5/), Saved Model) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite (HDF5, Saved Model)
<br></br>
## As an HDF5 file ##
Keras provides a basic save format using the HDF5 standard. For our purposes, the saved model can be treated as a single binary blob.

In [0]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 205us/sample - loss: 1.1541 - acc: 0.6790
Epoch 2/5
1000/1000 [==============================] - 0s 142us/sample - loss: 0.4188 - acc: 0.8820
Epoch 3/5
1000/1000 [==============================] - 0s 152us/sample - loss: 0.2950 - acc: 0.9290
Epoch 4/5
1000/1000 [==============================] - 0s 144us/sample - loss: 0.2049 - acc: 0.9530
Epoch 5/5
1000/1000 [==============================] - 0s 136us/sample - loss: 0.1588 - acc: 0.9680


In [0]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

W0818 03:50:35.178732 140051597113216 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0818 03:50:35.183529 140051597113216 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


This technique saves everything:
* The weight values
* The model's configuration(architecture)
* The optimizer configuration
<br></br>

Keras saves models by inspecting the architecture. Currently, it is not able to save TensorFlow optimizers (from ***tf.train***). When using those you will need to re-compile the model after loading, and you will lose the state of the optimizer.

# Create a saved model #

In [0]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 216us/sample - loss: 1.1910 - acc: 0.6730
Epoch 2/5
1000/1000 [==============================] - 0s 139us/sample - loss: 0.4325 - acc: 0.8770
Epoch 3/5
1000/1000 [==============================] - 0s 138us/sample - loss: 0.2910 - acc: 0.9190
Epoch 4/5
1000/1000 [==============================] - 0s 145us/sample - loss: 0.2123 - acc: 0.9480
Epoch 5/5
1000/1000 [==============================] - 0s 151us/sample - loss: 0.1627 - acc: 0.9700


In [0]:
import time

saved_model_path = "./saved_models/"+str(int(time.time()))
tf.contrib.saved_model.save_keras_model(model, saved_model_path)

W0818 03:58:01.616301 140051597113216 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0818 03:58:02.740245 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0818 03:58:02.742094 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0818 03:58:02.746789 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0818 03:58:02.750587 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0818 03:58:02.758590 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0818 03:58:02.760432 140051597113

In [0]:
!ls {saved_model_path}

assets	saved_model.pb	variables


Reload a fresh keras model from the saved model.

In [0]:
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model.summary()

W0818 03:58:51.996727 140051597113216 deprecation.py:323] From <ipython-input-21-f5ba023fc0b4>:1: load_from_saved_model (from tensorflow.python.keras.saving.saved_model_experimental) is deprecated and will be removed in a future version.
Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.
W0818 03:58:52.072551 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0818 03:58:52.073704 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0818 03:58:52.079524 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0818 03:58:52.081061 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0818 03:58:52.085735 140051597113216 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.kernel
W0818 03:58:52.090070 140051597113

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Run the restored model.

In [0]:
# The model has to be compiled before evaluating.
# This step is not required if the saved model is only being deployed.

new_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

# Evaluate the restored model.
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 105us/sample - loss: 0.4293 - acc: 0.8580
Restored model, accuracy: 85.80%


# Conclusion #
That was a quick guide to saving and loading in with tf.keras.

# Reference #

[Source](https://www.tensorflow.org/tutorials/keras/save_and_restore_models)

Prepared by: [Wong Jian Zhi](https://github.com/jianzhiw)

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.